In [47]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

In [48]:
# Starting dataset
df = pd.read_csv('cleaned_dataframe.csv', sep='\t', index_col=0)

In [49]:
df.head()

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
0,536365,01/12/10 08:26,2.55,17850,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,01/12/10 08:26,3.39,17850,United Kingdom,71053,WHITE METAL LANTERN,6
2,536365,01/12/10 08:26,2.75,17850,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8
3,536365,01/12/10 08:26,3.39,17850,United Kingdom,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,01/12/10 08:26,3.39,17850,United Kingdom,84029E,RED WOOLLY HOTTIE WHITE HEART.,6


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400220 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BasketID         400220 non-null  object 
 1   BasketDate       400220 non-null  object 
 2   Sale             400220 non-null  float64
 3   CustomerID       400220 non-null  int64  
 4   CustomerCountry  400220 non-null  object 
 5   ProdID           400220 non-null  object 
 6   ProdDescr        400220 non-null  object 
 7   Qta              400220 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 27.5+ MB


In [51]:
# Create the dataframe for customers
data = pd.core.frame.DataFrame({'CustomerID' : df['CustomerID'].unique()})

In [52]:
unique_products = []
for customer in df['CustomerID'].unique():
    unique_products.append(len(df.loc[df['CustomerID'] == customer, ['ProdID']].drop_duplicates()))

In [53]:
# Iu: the number of distinct items bought by a customer in the period of observation.
data['N_Dist_Prods'] = unique_products

In [54]:
# Imax: the maximum number of items purchased by a customer during a shopping session
data['Imax'] = 0

In [55]:
# E: the Shannon entropy on the purchasing behaviour of the customer
data['E'] = 0

In [56]:
# ProdPerOrderMean: mean number of items purchased by a customer during a shopping session

In [57]:
# SaleTot: total amount spent during the period of observation

In [58]:
# SaleTotWithoutReturn: total amount spent without negative quantity during the period of observation

In [59]:
# SaleMin: min amout spent during the shopping session

In [60]:
# SaleMeanPerOrder: mean amount spent for each order during the period of observation

In [61]:
# NumRetProd: number of returned products in the period of observation

In [62]:
# NumDistRetProd: number of distinct returned products in the period of observation

In [63]:
# MeanProdCostInOrder: mean cost of the products in the order

In [64]:
# OrderPerMonth

In [65]:
# ProductPerMonth

In [66]:
# PrizePerMonth

In [67]:
# OrderPerTrimester

In [68]:
# ProductPerTrimester

In [69]:
# PrizePerTrimester

In [70]:
# OrderPerTrimester 2-3

In [71]:
# ProductPerTrimester 2-3

In [72]:
# PrizePerTrimester 2-3

In [73]:
# CustomerCounty : value from original dataframe
#data['CustomerCountry'] = ''
#data = data.join(pd.core.frame.DataFrame({'CustomerID':df['CustomerID'], 'CustomerCountry':df['CustomerCountry']}).drop_duplicates().set_index('CustomerID'), on='CustomerID')

#tmp = pd.core.frame.DataFrame({'CustomerID':df['CustomerID'], 'CustomerCountry':df['CustomerCountry']}).drop_duplicates()
#tmp['CustomerID'].value_counts()[0:8]
# 4372 distinct customers - 4380 elements in tmp
# 8 customers have two different nationalities

# Find entries with multiple nationality
#counts = tmp['CustomerID'].value_counts().items()
#count_dict = {x[0]:x[1] for x in counts}
#for _, row in tmp.sort_values(by=['CustomerID']).iterrows():
#    if count_dict[row['CustomerID']] > 1:
#        print(f"{row['CustomerID']}-{row['CustomerCountry']}")

In [74]:
data

,CustomerID,N_Dist_Prods,Imax,E
0,17850,21,0,0
1,13047,106,0,0
2,12583,115,0,0
3,13748,24,0,0
4,15100,1,0,0
...,...,...,...,...
4334,13436,12,0,0
4335,15520,18,0,0
4336,13298,2,0,0
4337,14569,10,0,0
